# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

import statsmodels.api as sm
predictors_int = sm.add_constant(boston_features)
model = sm.OLS(boston['target'],predictors_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     109.7
Date:                Fri, 16 Aug 2019   Prob (F-statistic):          1.12e-130
Time:                        12:59:53   Log-Likelihood:                -1511.3
No. Observations:                 506   AIC:                             3049.
Df Residuals:                     493   BIC:                             3104.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             13.2225      2.449      5.400      0.000       8.411      18.034
CRIM              -0.0941      0.032     -2.912      0.004      -0.158      -0.031
INDUS             -0.1216      0.058     -2.098      0.036      -0.236      -0.008
CHAS               2.8363      0.881      3.220      0.001       1.106       4.567
RM                 3.9431      0.430      9.179      0.000       3.099       4.787
AGE               -0.0192      0.013     -1.474      0.141      -0.045       0.006
DIS               -0.8963      0.181     -4.955      0.000      -1.252      -0.541
PTRATIO           -0.8961      0.120     -7.446      0.000      -1.133      -0.660
B                  0.0103      0.003      3.771      0.000       0.005       0.016
LSTAT             -0.5373      0.052    -10.387      0.000      -0.639      -0.436
RAD_(0, 6]         5.5948      1.160      4.824      0.000       3.316       7.874
RAD_(6, 24]        7.6277      1.357      5.623      0.000       4.962      10.293
TAX_(0, 270]       5.8799      0.932      6.312      0.000       4.050       7.710
TAX_(270, 360]     3.5420      0.892      3.973      0.000       1.790       5.294
TAX_(360, 712]     3.8006      0.915      4.153      0.000       2.002       5.599
==============================================================================
Omnibus:                      189.311   Durbin-Watson:                   1.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              943.583
Skew:                           1.581   Prob(JB):                    1.27e-205
Kurtosis:                       8.896   Cond. No.                     3.60e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.5e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [14]:
from sklearn.linear_model import LinearRegression

y = boston['target']
linreg = LinearRegression()
linreg.fit(boston_features, y)

linreg.coef_

array([-0.0940753 , -0.12163231,  2.83627798,  3.94308668, -0.01915265,
       -0.89631217, -0.89613372,  0.01029422, -0.53731737, -1.01641032,
        1.01641032,  1.4723916 , -0.86550781, -0.60688379])

In [15]:
linreg.intercept_

24.241284230254532

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [17]:
predictors = boston_features.drop("TAX",axis=1)
predictors = 

linreg.fit(predictors, y)
linreg.coef_

KeyError: "['TAX'] not found in axis"

In [ ]:
l

### Statsmodels

### Scikit-learn

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.